In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

C:\Users\lovet\AppData\Local\Temp\ipykernel_20068\1410629650.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('US_Accidents_March23.csv')

In [3]:
df = df[['Severity', 'Start_Time', 'State', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Amenity', 
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop']]

In [5]:
df['Hour'] = pd.to_datetime(df['Start_Time'], errors='coerce').dt.hour
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)
df_model = df.drop(columns=['Start_Time'])
df_model = df_model.dropna()
df_model.head()

,Severity,State,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),...,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Hour
5,3,OH,37.9,35.5,97.0,29.63,7.0,SSW,3.5,0.03,...,0,0,0,0,0,0,0,0,0,7.0
9,3,OH,37.4,33.8,100.0,29.62,3.0,SSW,4.6,0.02,...,0,0,0,0,0,0,0,0,0,8.0
11,3,OH,37.4,33.8,100.0,29.62,3.0,SSW,4.6,0.02,...,0,0,0,0,0,0,0,0,0,8.0
14,2,OH,37.4,33.8,100.0,29.62,3.0,SSW,4.6,0.02,...,0,0,0,0,0,0,0,1,0,8.0
20,2,OH,33.8,29.6,100.0,29.62,2.0,NNW,4.6,0.01,...,0,0,0,0,0,0,0,0,0,10.0


In [7]:
print(df_model.columns)

Index(['Severity', 'State', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
       'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Amenity', 'Type_of_Road', 'Hour'],
      dtype='object')


In [6]:
df_model.to_csv('cleaned_traffic_data.csv', index=False)